In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os



In [2]:
data_path = os.path.join("..", "data")

In [3]:
X = np.load(data_path + '/initial_data/function_4/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_4/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.89698105 0.72562797 0.17540431 0.70169437]
 [0.8893564  0.49958786 0.53926886 0.50878344]
 [0.25094624 0.03369313 0.14538002 0.49493242]
 [0.34696206 0.0062504  0.76056361 0.61302356]
 [0.12487118 0.12977019 0.38440048 0.2870761 ]
 [0.80130271 0.50023109 0.70664456 0.19510284]
 [0.24770826 0.06044543 0.04218635 0.44132425]
 [0.74670224 0.7570915  0.36935306 0.20656628]
 [0.40066503 0.07257425 0.88676825 0.24384229]
 [0.6260706  0.58675126 0.43880578 0.77885769]
 [0.95713529 0.59764438 0.76611385 0.77620991]
 [0.73281243 0.14524998 0.47681272 0.13336573]
 [0.65511548 0.07239183 0.68715175 0.08151656]
 [0.21973443 0.83203134 0.48286416 0.08256923]
 [0.48859419 0.2119651  0.93917791 0.37619173]
 [0.16713049 0.87655456 0.21723954 0.95980098]
 [0.21691119 0.16608583 0.24137226 0.77006248]
 [0.38748784 0.80453226 0.75179548 0.72382744]
 [0.98562189 0.66693268 0.15678328 0.8565348 ]
 [0.03782483 0.66485335 0.16198218 0.25392378]
 [0.68348638 0.9027701  0.33541983 0.99948256]
 [0.1703

In [4]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.428571,0.448979,0.387755,0.448979]], axis=0)
y = np.append(y, [-0.281966339], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.469388, 0.469388, 0.285714, 0.428571]], axis=0)
y = np.append(y, [-2.5093632338739513], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.372475, 0.484827, 0.477273, 0.411341]], axis=0)
y = np.append(y, [-2.2686705479562543], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.310497, 0.337378, 0.332006, 0.378659]], axis=0)
y = np.append(y, [-0.8114645399106837], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.437681, 0.304730, 0.402971, 0.465417]], axis=0)
y = np.append(y, [-1.5478681188544452], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.296830, 0.339701, 0.328080, 0.381113]], axis=0)
y = np.append(y, [-1.2401877238019376], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.409933, 0.364803, 0.392926, 0.352123]], axis=0)
y = np.append(y, [0.5774975942545706], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.89698105 0.72562797 0.17540431 0.70169437]
 [0.8893564  0.49958786 0.53926886 0.50878344]
 [0.25094624 0.03369313 0.14538002 0.49493242]
 [0.34696206 0.0062504  0.76056361 0.61302356]
 [0.12487118 0.12977019 0.38440048 0.2870761 ]
 [0.80130271 0.50023109 0.70664456 0.19510284]
 [0.24770826 0.06044543 0.04218635 0.44132425]
 [0.74670224 0.7570915  0.36935306 0.20656628]
 [0.40066503 0.07257425 0.88676825 0.24384229]
 [0.6260706  0.58675126 0.43880578 0.77885769]
 [0.95713529 0.59764438 0.76611385 0.77620991]
 [0.73281243 0.14524998 0.47681272 0.13336573]
 [0.65511548 0.07239183 0.68715175 0.08151656]
 [0.21973443 0.83203134 0.48286416 0.08256923]
 [0.48859419 0.2119651  0.93917791 0.37619173]
 [0.16713049 0.87655456 0.21723954 0.95980098]
 [0.21691119 0.16608583 0.24137226 0.77006248]
 [0.38748784 0.80453226 0.75179548 0.72382744]
 [0.98562189 0.66693268 0.15678328 0.8565348 ]
 [0.03782483 0.66485335 0.16198218 0.25392378]
 [0.68348638 0.9027701  0.33541983 0.99948256]
 [0.1703

In [5]:
print(f"max y is :{np.max(y)}")

max y is :0.5774975942545706


In [6]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=2.5),
                         normalize_y=True)

In [7]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
# week 7 changed kappa to 1.0 from 4.0
# week 8 changed kappa to 4.0.5 from §.0 for more exploitation
def ucb(x, gp, kappa=0.5):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
"""grid_size = 50
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
x4 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c, d] for a in x1 for b in x2 for c in x3 for d in x4])"""

In [8]:
n_samples = 3000000 # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^6
X_random = np.random.rand(n_samples, 4)

In [9]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [10]:
best_idx = np.argmax(ucb_values)
next_best_point = X_random[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))    
print("Next best point to evaluate (UCB) without rounding:", next_best_point)  
print("Predicted maximum value (mean):", pred_mean[0])




Next best point to evaluate (UCB) rounded: [0.40228  0.380074 0.37591  0.363568]
Next best point to evaluate (UCB) without rounding: [0.40227985 0.38007438 0.37591014 0.36356768]
Predicted maximum value (mean): 0.645932014754834


In [11]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 4)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)

Best candidate input: [0.40298656 0.41053885 0.39897654 0.39841731]
Best candidate input rounded: [0.402987 0.410539 0.398977 0.398417]
Predicted maximum value: -0.0996219628347923
